TeloNP Demo + Combining Reads with Alignment Tables

In [1]:
from Bio import SeqIO
import sys
import os
import gzip
import numpy as np
import pandas as pd

sys.path.insert(0, '../TeloBP/')
from TeloBP import *
import constants as c

Loading in data

In [2]:
def getSampleKeyFromFilename(file):
    NBtitle = [x for x in file.split('.') if "NB" in x][0]
    NBtitle = [x for x in NBtitle.split('_') if "NB" in x][0]
    NBtitle = NBtitle.replace("uq", "")
    Ftitle = [x for x in file.split('.') if "F" in x][0]
    Ftitle = [x for x in Ftitle.split('_') if "F" in x][0]
    sampleKey = Ftitle + "_" + NBtitle
    return sampleKey

In [3]:
# Here we read in the tables which contain read qnames and alignment information

# sampleQnamesDir = "../../Data/sampleQnamesData/assignment/"
sampleQnamesDir = "C:/Users/Ramin Kahidi/Bioinformatics/Telomere Analysis/Nanopore project/Data/KarTongData/assignment/"

sampleQnames = {}

for root, dirs, files in os.walk(sampleQnamesDir):
    for file in files:
        if not file.endswith('.txt'):
            continue
        # skip empty files
        if os.stat(os.path.join(root, file)).st_size == 0:
            continue
        print(file)
        
        sampleKey = getSampleKeyFromFilename(file)
        print(sampleKey)
        # read in table and set columns
        sampleQnames[sampleKey] = pd.read_csv(os.path.join(root, file), delimiter='\t', header=None)
        sampleQnames[sampleKey].columns = ["qname", "seqLength", "sampleQnamesChr", "subTeloAlignLength"]


F33.JH39-3.NB50uq.fastq.mappan43.assignedarm.txt
F33_NB50
F33.JH70.NB01uq.fastq.mappan43.assignedarm.txt
F33_NB01
F33.JH71.NB72uq.fastq.mappan43.assignedarm.txt
F33_NB72
F38.all.JH113.NB67uq.fastq.mappan43.assignedarm.txt
F38_NB67
F38.all.JH122.NB69uq.fastq.mappan43.assignedarm.txt
F38_NB69
F39.all.JH123CB.NB70uq.fastq.mappan43.assignedarm.txt
F39_NB70
F39.all.JH124CB.NB72uq.fastq.mappan43.assignedarm.txt
F39_NB72
F40.all.JH76.NB01uq.fastq.mappan43.assignedarm.txt
F40_NB01
F40.all.JH77.NB15uq.fastq.mappan43.assignedarm.txt
F40_NB15
F41.2xtelo.JH80.NB50uq.fastq.mappan43.assignedarm.txt
F41_NB50
F41.2xtelo.JH81.NB65uq.fastq.mappan43.assignedarm.txt
F41_NB65
F42.2xtelo.JH82.NB67uq.fastq.mappan43.assignedarm.txt
F42_NB67
F42.2xtelo.JH88.NB69uq.fastq.mappan43.assignedarm.txt
F42_NB69
F43.2xtelo.JH89.NB70uq.fastq.mappan43.assignedarm.txt
F43_NB70
F43.2xtelo.JH90.NB69uq.fastq.mappan43.assignedarm.txt
F43_NB69
F44.all.JH91.NB72uq.fastq.mappan43.assignedarm.txt
F44_NB72
F44.all.JH93.NB01uq.fast

In [4]:
# This section will read in fastq files and extract the sequences for each read

# fastqReadDir = "../../Data/Final.demultip.tagged.fastq"
fastqReadDir = "C:/Users/Ramin Kahidi/Bioinformatics/Telomere Analysis/Nanopore project/Data/Final.demultip.tagged.fastq"

for root, dirs, files in os.walk(fastqReadDir):
    for filename in files:
        print(filename)
        if not filename.endswith(".gz") and not filename.endswith(".fastq") or "AG" in filename:
            continue

        sampleKey = getSampleKeyFromFilename(filename)
        print(sampleKey)

        if sampleKey not in sampleQnames.keys():
            continue
        print(f"Processing {filename}")
        sampleDf = sampleQnames[sampleKey]

        qnameTeloValues = []
        file = os.path.join(root, filename)
        print(file)
        if filename.endswith("fastq.gz"):
            with gzip.open(file,"rt") as handle:
                records = SeqIO.parse(handle,"fastq")
                for record in records:
                    if record.id not in sampleDf["qname"].tolist():
                        continue
                    qnameTeloValues.append([record.id, record.seq])
        elif filename.endswith("fastq"):
            for record in SeqIO.parse(file,"fastq"):
                if record.id not in sampleDf["qname"].tolist():
                    continue
                qnameTeloValues.append([record.id, record.seq])
        qnameTeloValuesDf = pd.DataFrame(qnameTeloValues, columns = ["qname", "seq"])
        sampleQnames[sampleKey] = pd.merge(sampleDf, qnameTeloValuesDf, on='qname', how='left')
        # ********************************************************************************************************************
        # Uncomment the following line when testing to break after the first file
        # break
        # ********************************************************************************************************************


AG1A24Mix3.NB50uq.fastq.gz
AG1A24Mix3.NB68uq.fastq.gz
AG1A24Mix3.NB88uq.fastq.gz
AG1A24Mix4.NB50uq.fastq.gz
AG1A24Mix4.NB68uq.fastq.gz
AG1A24Mix4.NB69uq.fastq.gz
AG1A24Mix4.NB88uq.fastq.gz
F33.JH39-3.NB50uq.fastq.gz
F33_NB50
Processing F33.JH39-3.NB50uq.fastq.gz
C:/Users/Ramin Kahidi/Bioinformatics/Telomere Analysis/Nanopore project/Data/Final.demultip.tagged.fastq\F33.JH39-3.NB50uq.fastq.gz
F33.JH70.NB01uq.fastq.gz
F33_NB01
Processing F33.JH70.NB01uq.fastq.gz
C:/Users/Ramin Kahidi/Bioinformatics/Telomere Analysis/Nanopore project/Data/Final.demultip.tagged.fastq\F33.JH70.NB01uq.fastq.gz
F33.JH71.NB72uq.fastq.gz
F33_NB72
Processing F33.JH71.NB72uq.fastq.gz
C:/Users/Ramin Kahidi/Bioinformatics/Telomere Analysis/Nanopore project/Data/Final.demultip.tagged.fastq\F33.JH71.NB72uq.fastq.gz
F35.2xtelo.JH78.NB01.v1.fastq.gz
F35_NB01
F35.2xtelo.JH83.NB15.v1.fastq.gz
F35_NB15
F36.all.JH84.NB19.fastq.gz
F36_NB19
F36.all.JH85.NB50.fastq.gz
F36_NB50
F37.all.JH86.NB65.fastq.gz
F37_NB65
F37.all.JH87.

Processing the data

In [5]:
# For each table in sampleQnames, remove rows with NaN in seq column
# This is because some reads may not have been present in the fastq files
popKeys = []
sampleQnamesNan = {}
for sampleKey in sampleQnames.keys():
    # print(sampleKey)
    sampleDf = sampleQnames[sampleKey]
    if "seq" not in sampleDf.keys():
        popKeys.append(sampleKey)
        continue
    # print(len(sampleDf))
    for index, row in sampleDf.iterrows():
        if row["seq"] is np.nan:
            if sampleKey not in sampleQnamesNan.keys():
                sampleQnamesNan[sampleKey] = [row]
            else:
                sampleQnamesNan[sampleKey].append(row)
            sampleDf.drop(index, inplace=True)

print(popKeys)
for key in popKeys:
    sampleQnames.pop(key)


[]


In [6]:
# Find any duplicated qnames in the tables:
# No output from this cell is good. It means there are no duplicated qnames.
for sampleKey in sampleQnames.keys():
    sampleDf = sampleQnames[sampleKey]
    # print out any duplicates in the qname column
    dupQnames = sampleDf[sampleDf.duplicated(['qname'])]["qname"].tolist()
    if len(dupQnames) > 0:
        # sort by qname
        print(sampleKey)
        sortedDf = sampleDf.sort_values(by=['qname'])
        print(sortedDf)

Run TeloBP on the data

In [7]:
from pandarallel import pandarallel

outputDir = "output/TeloNPInprogress"

def rowToTeloBP(row):
    import numpy as np
    from TeloBP import getTeloNPBoundary
    # This if statement is to catch any rows which have NaN in the seq column. 
    # Ideally this should not be necessary, but it is here just in case.
    if row["seq"] is np.nan:
        return -1000
    
    teloLength = getTeloNPBoundary(row["seq"])
    return teloLength

# # The following will multiprocess the rowToTeloBP function
# for sampleKey in sampleQnames.keys():
#     sampleDf = sampleQnames[sampleKey]
    
#     pandarallel.initialize(progress_bar=True )
#     sampleDf["teloBPLengths"] = sampleDf.parallel_apply(rowToTeloBP,axis=1)

#     # I highly recommend multi-processing, but if you want to single process,
#     # comment out the above two lines and uncomment the following line:
#     # sampleDf["teloBPLengths"] = sampleDf.apply(lambda row: rowToTeloBP(row), axis=1)

#     sampleQnames[sampleKey] = sampleDf

#     # save the output to a csv file. 
#     # Note that the seq column is removed from the table before saving 
#     sampleQnames[sampleKey] = sampleQnames[sampleKey].drop(columns=["seq"])
#     sampleQnames[sampleKey].to_csv(f"{outputDir}/{sampleKey}.csv")


# The following will multiprocess each table into its own process and save the output to a csv file


def process_table(sampleKey):
    sampleDf = sampleQnames[sampleKey]
    
    # Apply the function to each row
    sampleDf["teloBPLengths"] = sampleDf.apply(rowToTeloBP, axis=1)

    # Save the output to a csv file. 
    # Note that the seq column is removed from the table before saving 
    sampleDf = sampleDf.drop(columns=["seq"])
    sampleDf.to_csv(f"{outputDir}/{sampleKey}.csv")

# Create a pool of processes
with mp.Pool(mp.cpu_count()) as pool:
    pool.map(process_table, sampleQnames.keys())

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


...

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


Processing the TeloNP results (Much of this code can be reduced into a single cell, separated for clarity)

In [ ]:
# Some of the inputted tables have different column names for their "end.aln" column, like "JH39.1ug.NB88uq.end.aln", so this section will fix this
error()
for sampleKey in sampleQnames:
    if not ("end.aln" in sampleQnames[sampleKey].columns):
        # merge any columns containing end.aln
        colSearchList = [x for x in sampleQnames[sampleKey].columns if "end.aln" in x]
        if len(colSearchList) == 0:
            print(f"No end.aln column found in {sampleKey}")
            continue
        # merge
        sampleQnames[sampleKey]["end.aln"] = sampleQnames[sampleKey][colSearchList].max(axis=1)
        #drop old columns
        sampleQnames[sampleKey].drop(columns=colSearchList, inplace=True)

No end.aln column found in F33_NB50


In [ ]:
# TeloNP will return the length in bp from the end of the sequence to the telomere boundary, meaning it will include any 
# barcode or telotag sequence as telomere. Here we are adjusting for this. 

for sampleKey in sampleQnames:
    sampleDf = sampleQnames[sampleKey]
    if "end.aln" not in sampleDf.columns:
        print(f"No end.aln column found in {sampleKey}")
        continue
    sampleDf["end.aln + 10"] = sampleDf["end.aln"] + 10
    sampleDf["TeloNPCorrectedLength"] = sampleDf["teloBPLengths"] - sampleDf["end.aln + 10"]
    # if unnamed column exists, drop it
    if "Unnamed: 0" in sampleDf.columns:
        sampleDf.drop(columns=["Unnamed: 0"], inplace=True)

No end.aln column found in F33_NB50


In [ ]:
# Removing any rows with teloBPLengths < 0

uncorrectedLengthUsed = False 
for sampleKey in sampleQnames.keys():
    sampleDf = sampleQnames[sampleKey]
    if "TeloNPCorrectedLength" in sampleDf.columns:
        sampleDf = sampleDf[sampleDf["TeloNPCorrectedLength"] > 0]
    else:
        uncorrectedLengthUsed = True
        sampleDf = sampleDf[sampleDf["teloBPLengths"] > 0]
    sampleQnames[sampleKey] = sampleDf

if uncorrectedLengthUsed:
    print("Warning: TeloNP corrected lengths were not used. teloBPLengths < 0 were filtered instead.")

In [ ]:
# Save the processed files

outputDir = "output/TeloNPOutput"

for sampleKey in sampleQnames.keys():
    sampleQnames[sampleKey].to_csv(f"{outputDir}/{sampleKey}.csv")
